In [478]:
%matplotlib widget

import sys
sys.path.append("./../scripts")
import PandemiXFunctions as pf
import PandemiXModelFunctions as pmf


import numpy as np

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }

In [479]:
import importlib 
importlib.reload(pmf)

<module 'PandemiXModelFunctions' from 'c:\\Users\\rakrpe\\OneDrive - Roskilde Universitet\\Documents\\PandemiX\\GithubRepos\\PandemiX\\modelling\\PandemiXModelFunctions.py'>

In [480]:
# Setup model

# Parameters
gamma_I = 1/7
gamma_Y = gamma_I 
beta_I = 1.5*gamma_I 
# beta_I = 1.25*gamma_I 
beta_Y = 3*gamma_Y
allPars = [beta_I,gamma_I,beta_Y,gamma_Y]

# Initial conditions
I_0 = 0.001
Y_0 = 0
R_0 = 0
S_0 = 1 - R_0 - I_0 - Y_0
init = [S_0,I_0,Y_0]

tEnd = 1000
curScheme = pmf.Scheme('SIYR',init,allPars,0,tEnd)

In [481]:

# d1 = 150
# d2 = 220
d1 = 90
d2 = 130
d3 = 500
newInf = 0.001
# Change_Add1 = pmf.Change(d3,AddVariables=[-newInf,0,newInf])
curAdd = {'S':-newInf,'I':0,'Y':newInf}
Change_Add1 = pmf.Change(d3,AddVariables=curAdd)
curScheme.addChange(Change_Add1)

In [482]:
print(curScheme)

-------
Model: SIYR.
Complete simulation running from t=0 until t=1000
Initial conditions: [0.999, 0.001, 0]
Parameters: [0.21428571428571427, 0.14285714285714285, 0.42857142857142855, 0.14285714285714285]
---
Changes: 
Change 0: At time 500, add S: -0.001, I: 0, Y: 0.001 to variables
---


In [483]:
# curScheme.ModelMeta
len(curAdd)

3

In [484]:
# Base simulation
curScheme.simulate(tRes=1000)

resBase = curScheme.result 
ts = resBase.t
Ss = resBase.y[0,:]
Is = resBase.y[1,:]
Ys = resBase.y[2,:]
Rs = 1 - Ss - Is - Ys

In [485]:
sch1 = curScheme.copy()



# betaMultiplier = 1.25
betaMultiplier = 2
RestrictionChange = pmf.Change(d1,MultiplyParameters=[betaMultiplier,1,betaMultiplier,1])
sch1.addChange(RestrictionChange)

sch2 = curScheme.copy()
RestrictionChange = pmf.Change(d2,MultiplyParameters=[betaMultiplier,1,betaMultiplier,1])
sch2.addChange(RestrictionChange)

sch1.sortChanges()
sch2.sortChanges()

sch1.simulate(tRes=1000)
sch2.simulate(tRes=1000)


ts1 = sch1.result.t
Ss1 = sch1.result.y[0,:]
Is1 = sch1.result.y[1,:]
Ys1 = sch1.result.y[2,:]
Rs1 = 1 - Ss1 - Is1 - Ys1

ts2 = sch2.result.t
Ss2 = sch2.result.y[0,:]
Is2 = sch2.result.y[1,:]
Ys2 = sch2.result.y[2,:]
Rs2 = 1 - Ss2 - Is2 - Ys2

In [486]:


fig,(ax1,ax2,ax3) = plt.subplots(3,1)

ax1.plot(ts,Rs)
ax1.plot(ts1,Rs1,'--')
ax1.plot(ts2,Rs2,':')

ax2.plot(ts,Is)
ax2.plot(ts1,Is1,'--')
ax2.plot(ts2,Is2,':')

ax3.plot(ts,Ys)
ax3.plot(ts1,Ys1,'--')
ax3.plot(ts2,Ys2,':')

ax1.axvline(d1,color='k',linestyle=':')
ax1.axvline(d2,color='k',linestyle=':')
ax1.axvline(d3,color='k',linestyle=':')
ax2.axvline(d1,color='k',linestyle=':')
ax2.axvline(d2,color='k',linestyle=':')
ax2.axvline(d3,color='k',linestyle=':')
ax3.axvline(d1,color='k',linestyle=':')
ax3.axvline(d2,color='k',linestyle=':')
ax3.axvline(d3,color='k',linestyle=':')


ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0)
ax3.set_ylim(bottom=0)

ax1.set_ylabel('Immune (R)')
ax2.set_ylabel('Variant 1 (I)')
ax3.set_ylabel('Variant 2 (Y)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Variant 2 (Y)')

In [487]:
# fig = plt.figure()
# (fig.get_size_inches()*fig.dpi)[1]
# len(fig.get_axes())

In [488]:
# repr(betaMultiplier* beta_I/gamma_I)
# betaMultiplier* beta_I/gamma_I
fig,allAxes = curScheme.plot(describeChanges=False)
fig,allAxes = sch1.plot(fig,color='b',linestyle=':',describeChanges=False)
fig,allAxes = sch2.plot(fig,color='m',linestyle='--',describeChanges=False)

for ax in allAxes:
    ax.set_ylim(bottom=0)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …